In [569]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import fileinput, glob, os
import datetime as dt
import missingno as msno
import seaborn as sns
import talib

In [149]:
#Import from stock_list.csv, filter by Universe=True and parse columns with dates:
stock_list = pd.read_csv('files/stock_list.csv', index_col=0, parse_dates=[1])
stock_list_incl = stock_list[stock_list.Universe0 == True]
stock_list_incl.TradeDate = pd.to_datetime(stock_list_incl.TradeDate, format='%Y%m%d')
stock_list_incl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1301 to 9997
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   EffectiveDate           2000 non-null   datetime64[ns]
 1   Name                    2000 non-null   object        
 2   Section/Products        2000 non-null   object        
 3   NewMarketSegment        2000 non-null   object        
 4   33SectorCode            2000 non-null   object        
 5   33SectorName            2000 non-null   object        
 6   17SectorCode            2000 non-null   object        
 7   17SectorName            2000 non-null   object        
 8   NewIndexSeriesSizeCode  2000 non-null   object        
 9   NewIndexSeriesSize      2000 non-null   object        
 10  TradeDate               2000 non-null   datetime64[ns]
 11  Close                   2000 non-null   float64       
 12  IssuedShares            2000 non-null   float

/Users/koytoora/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [150]:
stock_list_incl.head()

,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
SecuritiesCode,,,,,,,,,,,,,,,
1301,2021-12-30,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,2021-12-30,3080.0,10928283.0,3.365911e+10,True
1332,2021-12-30,"Nippon Suisan Kaisha,Ltd.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,4,TOPIX Mid400,2021-12-30,543.0,312430277.0,1.696496e+11,True
1333,2021-12-30,Maruha Nichiro Corporation,First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,4,TOPIX Mid400,2021-12-30,2405.0,52656910.0,1.266399e+11,True
1375,2021-12-30,"YUKIGUNI MAITAKE CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,6,TOPIX Small 1,2021-12-30,1196.0,39910700.0,4.773320e+10,True
1376,2021-12-30,"KANEKO SEEDS CO.,LTD.",First Section (Domestic),Standard Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,2021-12-30,1504.0,11772626.0,1.770603e+10,True


In [563]:
#Import from stock_prices.csv and parse columns with dates:
stock_prices = pd.read_csv('files/train_files/stock_prices.csv', index_col=0, parse_dates=[1])


In [564]:
stock_prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2332531 entries, 20170104_1301 to 20211203_9997
Data columns (total 11 columns):
 #   Column            Dtype         
---  ------            -----         
 0   Date              datetime64[ns]
 1   SecuritiesCode    int64         
 2   Open              float64       
 3   High              float64       
 4   Low               float64       
 5   Close             float64       
 6   Volume            int64         
 7   AdjustmentFactor  float64       
 8   ExpectedDividend  float64       
 9   SupervisionFlag   bool          
 10  Target            float64       
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2)
memory usage: 198.0+ MB


In [565]:
stock_prices.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
RowId,,,,,,,,,,,
20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [153]:
#Import from options.csv and parse columns with dates:
options = pd.read_csv('files/train_files/options.csv', index_col=0, parse_dates=[1],
                      converters={7:str,8:str,9:str,10:str})

In [154]:
#Correction of data types in some columns
options.LastTradingDay = pd.to_datetime(options.LastTradingDay, format='%Y%m%d')
options.SpecialQuotationDay = pd.to_datetime(options.SpecialQuotationDay, format='%Y%m%d')
options.NightSessionOpen = options.NightSessionOpen.str.replace('-','0')
options.NightSessionOpen = options.NightSessionOpen.astype('float')
options.NightSessionHigh = options.NightSessionHigh.str.replace('-','0')
options.NightSessionHigh = options.NightSessionHigh.astype('float')
options.NightSessionLow = options.NightSessionLow.str.replace('-','0')
options.NightSessionLow = options.NightSessionLow.astype('float')
options.NightSessionClose = options.NightSessionClose.str.replace('-','0')
options.NightSessionClose = options.NightSessionClose.astype('float')

In [155]:
options.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3567694 entries, 20170104_132010018 to 20211203_199248018
Data columns (total 30 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Date                 datetime64[ns]
 1   OptionsCode          int64         
 2   WholeDayOpen         float64       
 3   WholeDayHigh         float64       
 4   WholeDayLow          float64       
 5   WholeDayClose        float64       
 6   NightSessionOpen     float64       
 7   NightSessionHigh     float64       
 8   NightSessionLow      float64       
 9   NightSessionClose    float64       
 10  DaySessionOpen       float64       
 11  DaySessionHigh       float64       
 12  DaySessionLow        float64       
 13  DaySessionClose      float64       
 14  TradingVolume        int64         
 15  OpenInterest         int64         
 16  TradingValue         int64         
 17  ContractMonth        int64         
 18  StrikePrice          float64       
 19

In [156]:
#Import from trades.csv, drop NaNs, and parse columns with dates:
trades = pd.read_csv('files/train_files/trades.csv', parse_dates=[0,1,2])
trades_full = trades.dropna()

In [157]:
trades_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765 entries, 6 to 1710
Data columns (total 56 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Date                                 765 non-null    datetime64[ns]
 1   StartDate                            765 non-null    datetime64[ns]
 2   EndDate                              765 non-null    datetime64[ns]
 3   Section                              765 non-null    object        
 4   TotalSales                           765 non-null    float64       
 5   TotalPurchases                       765 non-null    float64       
 6   TotalTotal                           765 non-null    float64       
 7   TotalBalance                         765 non-null    float64       
 8   ProprietarySales                     765 non-null    float64       
 9   ProprietaryPurchases                 765 non-null    float64       
 10  ProprietaryTo

In [158]:
#Import from financials.csv and parse dates:
financials = pd.read_csv('files/train_files/financials.csv', index_col=0, parse_dates=[2,4,8,10,11],
                        converters={14:str,15:str,16:str,17:str,18:str,19:str,42:str,44:str})

In [159]:
financials.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 92956 entries, 20161207453651.0 to 20211203446277.0
Data columns (total 44 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   DateCode                                                                      92954 non-null  object        
 1   Date                                                                          92956 non-null  datetime64[ns]
 2   SecuritiesCode                                                                92954 non-null  float64       
 3   DisclosedDate                                                                 92954 non-null  datetime64[ns]
 4   DisclosedTime                                                                 92954 non-null  object        
 5   DisclosedUnixTime                                           

In [566]:
#Это бесполезные операции, сначала генерирурю список дней и акций, а потом джойном удаляю строки, для которых нет данных в stock_prices
#Просто потренировался, можно сразу брать индексы из stock_prices
date_range = pd.date_range(stock_prices.Date.min(), stock_prices.Date.max(), freq='D')
dates = np.array(date_range)
stocks = np.array(stock_list_incl.index)
multi_index = pd.MultiIndex.from_product([dates, stocks], names=['Date','SecuritiesCode'])
final_full = multi_index.to_frame(index = False)
final1 = final_full.merge(stock_prices.iloc[:,:9], how='inner', on=['Date','SecuritiesCode'])
final1

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN
...,...,...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN
2332527,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN
2332528,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN
2332529,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN


In [557]:
#Скользящее среднее за 50 дней
sma50 = final1.set_index('Date').groupby(['SecuritiesCode'])['Close'].rolling(window='50D').mean()
sma50.name = 'SMA50'
final2 = final1.merge(sma50, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
#Скользящее среднее за 10 дней
sma10 = final2.set_index('Date').groupby(['SecuritiesCode'])['Close'].rolling(window='10D').mean()
sma10.name = 'SMA10'
final3 = final2.merge(sma10, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
#Относительная разница цен закрытия и скользящих средних
final3['SMA50_Close_diff'] = (final3.Close-final3.SMA50) / final3.Close
final3['SMA10_Close_diff'] = (final3.Close-final3.SMA10) / final3.Close
final3

,Date,SecuritiesCode,Open,High,Low,Close,Volume,SMA50,SMA10,SMA50_Close_diff,SMA10_Close_diff
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,2742.000000,2742.000,0.000000,0.000000
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,571.000000,571.000,0.000000,0.000000
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,3210.000000,3210.000,0.000000,0.000000
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1550.000000,1550.000,0.000000,0.000000
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,3330.000000,3330.000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,563.794118,532.500,-0.067792,-0.008523
2332527,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,831.617647,787.875,-0.047377,0.007714
2332528,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1708.117647,1680.625,-0.038369,-0.021657
2332529,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,2404.852941,2370.125,-0.006636,0.007901


In [558]:
#Доля объемов на акцию от общедневного объема торговли
temp1 = final3.groupby('Date').agg('sum')['Volume']
final3 = final3.rename(columns={'Volume': 'ind_volume'})
final4 = final3.merge(temp1, how='left', left_on='Date', right_index=True)
final4 = final4.rename(columns={'Volume': 'tot_volume'})
final4['Volume_share'] = final4['ind_volume'] / final4['tot_volume']
final4

,Date,SecuritiesCode,Open,High,Low,Close,ind_volume,SMA50,SMA10,SMA50_Close_diff,SMA10_Close_diff,tot_volume,Volume_share
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,2742.000000,2742.000,0.000000,0.000000,2334312600,0.000013
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,571.000000,571.000,0.000000,0.000000,2334312600,0.001199
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,3210.000000,3210.000,0.000000,0.000000,2334312600,0.000116
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1550.000000,1550.000,0.000000,0.000000,2334312600,0.000005
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,3330.000000,3330.000,0.000000,0.000000,2334312600,0.000065
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,563.794118,532.500,-0.067792,-0.008523,1238819500,0.000036
2332527,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,831.617647,787.875,-0.047377,0.007714,1238819500,0.000029
2332528,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1708.117647,1680.625,-0.038369,-0.021657,1238819500,0.000006
2332529,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,2404.852941,2370.125,-0.006636,0.007901,1238819500,0.000005


In [559]:
#Полосы Боллинджера:
#средняя линия уже есть, это SMA10
#верхняя линия: средняя линия + (10 * стандартное отклонение цен закрытия за 10 дней)
#нижняя линия: средняя линия - (10 * стандартное отклонение цен закрытия за 10 дней)
sma10stdev = final4.set_index('Date').groupby(['SecuritiesCode'])['Close'].rolling(window='10D').std()
sma10stdev.name = 'SD10'
final5 = final4.merge(sma10stdev, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final5['Upper_line'] = final5['SMA10'] + final5['SD10']
final5['Lower_line'] = final5['SMA10'] - final5['SD10']
#относительная разница между ценой закрытия и нижней/верхней линиями
final5['UL_Close-diff'] = (final5.Close-final5.Upper_line) / final5.Close
final5['LL_Close-diff'] = (final5.Close-final5.Lower_line) / final5.Close
final5

,Date,SecuritiesCode,Open,High,Low,Close,ind_volume,SMA50,SMA10,SMA50_Close_diff,SMA10_Close_diff,tot_volume,Volume_share,SD10,Upper_line,Lower_line,UL_Close-diff,LL_Close-diff
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,2742.000000,2742.000,0.000000,0.000000,2334312600,0.000013,NaN,NaN,NaN,NaN,NaN
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,571.000000,571.000,0.000000,0.000000,2334312600,0.001199,NaN,NaN,NaN,NaN,NaN
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,3210.000000,3210.000,0.000000,0.000000,2334312600,0.000116,NaN,NaN,NaN,NaN,NaN
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1550.000000,1550.000,0.000000,0.000000,2334312600,0.000005,NaN,NaN,NaN,NaN,NaN
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,3330.000000,3330.000,0.000000,0.000000,2334312600,0.000065,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,563.794118,532.500,-0.067792,-0.008523,1238819500,0.000036,17.435596,549.935596,515.064404,-0.041545,0.024499
2332527,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,831.617647,787.875,-0.047377,0.007714,1238819500,0.000029,18.011405,805.886405,769.863595,-0.014970,0.030398
2332528,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1708.117647,1680.625,-0.038369,-0.021657,1238819500,0.000006,26.262072,1706.887072,1654.362928,-0.037621,-0.005692
2332529,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,2404.852941,2370.125,-0.006636,0.007901,1238819500,0.000005,27.518500,2397.643500,2342.606500,-0.003618,0.019420


In [574]:
#close = final5.set_index('Date').groupby(['SecuritiesCode'])['Close']
#rsi = talib.RSI(close, timeperiod=14)
#rsi

TypeError: Argument 'real' has incorrect type (expected numpy.ndarray, got SeriesGroupBy)

In [560]:
#RSI
#а) среднее значение последних 14 растущих дневных свечей
#б) среднее значение последних 14 падающих дневных свечей
#в) считаем RS: делим (а) на (б)
#г) считаем RSI: 100-(100/(1+RS))
final5["CloseUp"] = final5["Close"].where((final5["Close"] >= final5["Open"]), np.nan)
final5["CloseDown"] = final5["Close"].where((final5["Close"] <= final5["Open"]), np.nan)

last14up = final5.set_index('Date').groupby(['SecuritiesCode'])['CloseUp'].rolling(window=14, min_periods=1).mean()
last14up.name = 'SMA14UP'
final6 = final5.merge(last14up, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
last14down = final6.set_index('Date').groupby(['SecuritiesCode'])['CloseDown'].rolling(window=14, min_periods=1).mean()
last14down.name = 'SMA14DOWN'
final7 = final6.merge(last14down, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final7['RSI'] = (100-(100/(1+(final7.SMA14UP / final7.SMA14DOWN))))/100
final7


,Date,SecuritiesCode,Open,High,Low,Close,ind_volume,SMA50,SMA10,SMA50_Close_diff,...,SD10,Upper_line,Lower_line,UL_Close-diff,LL_Close-diff,CloseUp,CloseDown,SMA14UP,SMA14DOWN,RSI
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,2742.000000,2742.000,0.000000,...,NaN,NaN,NaN,NaN,NaN,2742.0,NaN,2742.000000,NaN,NaN
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,571.000000,571.000,0.000000,...,NaN,NaN,NaN,NaN,NaN,571.0,NaN,571.000000,NaN,NaN
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,3210.000000,3210.000,0.000000,...,NaN,NaN,NaN,NaN,NaN,3210.0,NaN,3210.000000,NaN,NaN
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1550.000000,1550.000,0.000000,...,NaN,NaN,NaN,NaN,NaN,1550.0,NaN,1550.000000,NaN,NaN
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,3330.000000,3330.000,0.000000,...,NaN,NaN,NaN,NaN,NaN,3330.0,NaN,3330.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,563.794118,532.500,-0.067792,...,17.435596,549.935596,515.064404,-0.041545,0.024499,528.0,NaN,548.333333,551.000000,0.498787
2332527,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,831.617647,787.875,-0.047377,...,18.011405,805.886405,769.863595,-0.014970,0.030398,794.0,NaN,798.000000,794.125000,0.501217
2332528,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1708.117647,1680.625,-0.038369,...,26.262072,1706.887072,1654.362928,-0.037621,-0.005692,NaN,1645.0,1717.250000,1694.333333,0.503359
2332529,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,2404.852941,2370.125,-0.006636,...,27.518500,2397.643500,2342.606500,-0.003618,0.019420,2389.0,NaN,2388.333333,2386.500000,0.500192


In [561]:
#Стохастический
#а) k = (цена закрытия - мин. цена дня)/(макс. цена дня - мин. цена дня)
#б) d = усредненная k за 3 дн
#теоретически возможно, что в знаменателе будет 0 три дня подряд и среднее не посчитается,
#на практике за 5 лет это не произошло ни разу, поэтому обрабатывать такое исключение не буду
final7["k"] = (final7["Close"]-final7["Low"])/((final7["High"]-final7["Low"]))
stohastic = final7.set_index('Date').groupby(['SecuritiesCode'])['k'].rolling(window=3, min_periods=1).mean()
stohastic.name = 'Stochastic'
final8 = final7.merge(stohastic, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final8

,Date,SecuritiesCode,Open,High,Low,Close,ind_volume,SMA50,SMA10,SMA50_Close_diff,...,Lower_line,UL_Close-diff,LL_Close-diff,CloseUp,CloseDown,SMA14UP,SMA14DOWN,RSI,k,Stochastic
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,2742.000000,2742.000,0.000000,...,NaN,NaN,NaN,2742.0,NaN,2742.000000,NaN,NaN,0.480000,4.800000e-01
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,571.000000,571.000,0.000000,...,NaN,NaN,NaN,571.0,NaN,571.000000,NaN,NaN,0.615385,6.153846e-01
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,3210.000000,3210.000,0.000000,...,NaN,NaN,NaN,3210.0,NaN,3210.000000,NaN,NaN,1.000000,1.000000e+00
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1550.000000,1550.000,0.000000,...,NaN,NaN,NaN,1550.0,NaN,1550.000000,NaN,NaN,1.000000,1.000000e+00
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,3330.000000,3330.000,0.000000,...,NaN,NaN,NaN,3330.0,NaN,3330.000000,NaN,NaN,0.750000,7.500000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,563.794118,532.500,-0.067792,...,515.064404,-0.041545,0.024499,528.0,NaN,548.333333,551.000000,0.498787,1.000000,4.848485e-01
2332527,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,831.617647,787.875,-0.047377,...,769.863595,-0.014970,0.030398,794.0,NaN,798.000000,794.125000,0.501217,1.000000,4.983974e-01
2332528,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1708.117647,1680.625,-0.038369,...,1654.362928,-0.037621,-0.005692,NaN,1645.0,1717.250000,1694.333333,0.503359,0.000000,4.331350e-13
2332529,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,2404.852941,2370.125,-0.006636,...,2342.606500,-0.003618,0.019420,2389.0,NaN,2388.333333,2386.500000,0.500192,0.562500,5.487698e-01


In [562]:
#Ебанина
#Индикатор схождения-расхождения скользящих средних (MACD)
# а) Экспоненциальная скользящая средняя за 12 дней: EMA = Close * 2/(n+1) + EMA(prev) * (1–2/(n+1))
# б) Экспоненциальная скользящая средняя за 26 дней: EMA = Close * 2/(n+1) + EMA(prev) * (1–2/(n+1))
# в) MACD = ЕМА_12 − EMA_26
# г) Экспоненциальная скользящая средняя за 9 дней: EMA = MACD * 2/(n+1) + MACD(prev) * (1–2/(n+1))
# е) разница между EMA9 и MACD

ema12 = final8.set_index('Date').groupby(['SecuritiesCode'])['Close'].ewm(span=12, adjust=False).mean()
ema12.name = 'EMA12'
ema26 = final8.set_index('Date').groupby(['SecuritiesCode'])['Close'].ewm(span=26, adjust=False).mean()
ema26.name = 'EMA26'
final9 = final8.merge(ema12, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final10 = final9.merge(ema26, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final10['MACD'] = final12['EMA12'] - final12['EMA26']
ema9 = final10.set_index('Date').groupby(['SecuritiesCode'])['MACD'].ewm(span=9, adjust=False).mean()
ema9.name = 'EMA9'
final11 = final10.merge(ema9, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final11['EMA_diff'] = (final11['EMA9'] - final11['MACD']) / final11['MACD']
final11

,Date,SecuritiesCode,Open,High,Low,Close,ind_volume,SMA50,SMA10,SMA50_Close_diff,...,SMA14UP,SMA14DOWN,RSI,k,Stochastic,EMA12,EMA26,MACD,EMA9,EMA_diff
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,2742.000000,2742.000,0.000000,...,2742.000000,NaN,NaN,0.480000,4.800000e-01,2742.000000,2742.000000,0.000000,0.000000,NaN
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,571.000000,571.000,0.000000,...,571.000000,NaN,NaN,0.615385,6.153846e-01,571.000000,571.000000,0.000000,0.000000,NaN
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,3210.000000,3210.000,0.000000,...,3210.000000,NaN,NaN,1.000000,1.000000e+00,3210.000000,3210.000000,0.000000,0.000000,NaN
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1550.000000,1550.000,0.000000,...,1550.000000,NaN,NaN,1.000000,1.000000e+00,1550.000000,1550.000000,0.000000,0.000000,NaN
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,3330.000000,3330.000,0.000000,...,3330.000000,NaN,NaN,0.750000,7.500000e-01,3330.000000,3330.000000,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,563.794118,532.500,-0.067792,...,548.333333,551.000000,0.498787,1.000000,4.848485e-01,540.283521,554.209490,-13.925969,-7.568447,-0.456523
2332527,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,831.617647,787.875,-0.047377,...,798.000000,794.125000,0.501217,1.000000,4.983974e-01,792.967132,813.304832,-20.337700,-20.271023,-0.003279
2332528,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1708.117647,1680.625,-0.038369,...,1717.250000,1694.333333,0.503359,0.000000,4.331350e-13,1685.476587,1700.961563,-15.484976,-7.578197,-0.510610
2332529,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,2404.852941,2370.125,-0.006636,...,2388.333333,2386.500000,0.500192,0.562500,5.487698e-01,2378.841847,2395.902470,-17.060623,-14.205326,-0.167362
